In [89]:
import numpy as np
import pandas as pd
import dabl
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

#Importing the train and test set
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

#Encoding the Sex feature
le = LabelEncoder() 
train_data['Sex'] = le.fit_transform(train_data['Sex']) 
test_data['Sex']  = le.fit_transform(test_data['Sex']) 

for dataset in [train_data, test_data]:
    #Filling Embarked null values with the mode
    dataset['Embarked'] = dataset['Embarked'].fillna(train_data.Embarked.dropna().mode()[0])
    #Summing over the family member (+1 refers to the passeger himself)
    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1
    #initialize to yes/1 is alone
    dataset['IsAlone'] = 1 
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0  
    
#Using the OneHotEncoder for the Embarked column
embarked_ohe = OneHotEncoder()    

X_TRAIN = embarked_ohe.fit_transform(train_data.Embarked.values.reshape(-1,1)).toarray()
X_TEST = embarked_ohe.fit_transform(test_data.Embarked.values.reshape(-1,1)).toarray()

dfOneHot = pd.DataFrame(X_TRAIN, columns = ["Embarked_"+str(int(i)) for i in range(X_TRAIN.shape[1])])
dfOneHot_TEST = pd.DataFrame(X_TEST, columns = ["Embarked_"+str(int(i)) for i in range(X_TEST.shape[1])])
train_data = pd.concat([train_data, dfOneHot], axis=1)
test_data = pd.concat([test_data, dfOneHot_TEST], axis=1)


#Filling Age and Fare NAN
train_data['Age'].fillna(train_data['Age'].dropna().median(), inplace=True)
train_data['Fare'].fillna(train_data['Fare'].dropna().median(), inplace=True)
test_data['Age'].fillna(test_data['Age'].dropna().median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].dropna().median(), inplace=True)

#Dropping unneeded columns
train_data = train_data.drop(['PassengerId','Ticket','Name','Cabin','Embarked'], axis=1)
test_data = test_data.drop(['Ticket','Name','Cabin','Embarked'], axis=1)

C:\Users\akhmi\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [90]:
#Fit and predict
train_data_y = train_data["Survived"]
model = RandomForestClassifier(n_estimators=120, max_depth=10, random_state=1)
model.fit(train_data.drop(['Survived'], axis=1), train_data_y)
predictions = model.predict(test_data.drop('PassengerId', axis=1))

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)